In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.file.ROI_reader import ROIFileReader

####################################################################################
# Input (organized into subfolders by date): 
#     ROI files (single cell responses) for each slice and location
#     Stimulation times in stim_times.csv
#     ZDA files
#     paired_pulse_summary.csv
# Output:
#     paired_pulse_summary_edited.csv (values for each ROI, ROI pixel count)
####################################################################################

In [2]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/Anna's/"
date_format = 'yyyy-mm-dd'
# measure window settings ( in frames )
pre_window_offset = 0
window_width = 40

input_csv = data_dir + 'paired_pulse_summary.csv'
output_csv = data_dir + 'paired_pulse_summary_edited.csv'
overwrite_intermed_dat_files = False  # whether to re-save value files

enable_photoZ_interaction = True
initialize_photoZ = False

# load the all-data CSV dir
df = pd.read_csv(input_csv)
df = pd.DataFrame(df)

In [3]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()

In [16]:
is_measure_window_width_set = window_width
for subdir, dirs, files in os.walk(data_dir):
    zda_files = []
    roi_files = {}  # map slice number to ROIs file
    
    if 'Usable' not in subdir:
        continue
    
    # locate important file names
    for f in files:
        if f.endswith(".zda"):
            zda_files.append(f)
        if 'ROIs' in f[-22:] and f.endswith('.dat'):
            metadata = f.replace('.dat','').split("_ROIs_")
            slice_no, loc_no = [int(x) for x in metadata[0].split("-")]
            if slice_no not in roi_files:
                roi_files[slice_no] = {}
            if loc_no not in roi_files[slice_no]:
                roi_files[slice_no][loc_no] = []
            start, end = [int(x) for x in metadata[1].split("_to_")]
            roi_files[slice_no][loc_no].append(
                {"start": start,
                "end": end,
                "filename": f}
            )
    
    # open stimtimes.csv file for this subdir
    stim_times_df = pd.read_csv(subdir + "/stimtimes.csv",
                                header=None,
                                names=['Rec_ID', 'Stim_Time_1', 'Stim_Time_2'])
   
    if len(roi_files) < 1 or len(zda_files) < 1:
        continue
    
    for zda_file in zda_files:
        rec_id = zda_file.split('.')[0]
            
        # determine date and slice, loc, rec
        aPhz = AutoPhotoZ(data_dir=subdir)
        trace_values_to_save = {
            'Max Amp': {'method': aPhz.select_maxamp_trace_value, 
                         'filename_id': "maxAmp_"},
            'Max Amp / SD': {'method': aPhz.select_SNR_trace_value, 
                         'filename_id': 'SNR_'},
            'Half Amp Latency': {'method': aPhz.select_latency_trace_value, 
                         'filename_id': "latency_"},
            'Half Width': {'method': aPhz.select_half_width_trace_value, 
                         'filename_id': 'halfWidth_'},
            'Max Amp Latency': {'method': aPhz.select_peaktime_trace_value, 
                         'filename_id': 'maxAmpLatency_'},
            'Half Rise Time': {'method': aPhz.select_half_rise_time_trace_value, 
                         'filename_id': 'riseTime_'},
            'Half Decay Time': {'method': aPhz.select_half_decay_time_trace_value, 
                         'filename_id': 'decayTime_'}
        }

        is_zda_file_open = False
        
        date = subdir.split("_Usable")[0][-10:]
        date = [int(x) for x in date.split("-")]
        if date_format != 'yyyy-mm-dd':
            date[2] += 2000  # full year format
        else:
            date = [date[1], date[2], date[0]]
        date = "/".join([str(d) for d in date])
        
        slice_no, loc_no, rec_no = [int(x) for x in rec_id.split("_")]
        
        # find stim times for this recording
        stim_times_rec = stim_times_df[stim_times_df['Rec_ID'] == rec_id]
        stim_time_1 = int(stim_times_rec['Stim_Time_1'])
        stim_time_2 = int(stim_times_rec['Stim_Time_2'])
        
        for roi_grp_idx in range(len(roi_files[slice_no][loc_no])):
            roi_file = roi_files[slice_no][loc_no][roi_grp_idx]['filename']
            roi_id_start = roi_files[slice_no][loc_no][roi_grp_idx]['start']
            roi_id_end = roi_files[slice_no][loc_no][roi_grp_idx]['end']
            print("\n\nscraping data from ", zda_file, 
                  "using", roi_file, "on date:", date)

            filename_end = rec_id + '.dat'
             
            # find # pixels in ROI 
            rfr = ROIFileReader(subdir + "/" + roi_file)
            n_pixels = [len(r) for r in rfr.get_roi_list()]
            
            # see if we can skip this entire ROI group (if files already exist)
            if not overwrite_intermed_dat_files:
                are_files_complete = True
                for trace_val_type in trace_values_to_save:
                    for stim_time_idx in range(1,3):
                        value_filename = subdir + "/" + trace_values_to_save[trace_val_type]['filename_id'] \
                                                + str(stim_time_idx) + "_" + filename_end
                        # may be able to skip saving file
                        if not os.path.exists(value_filename):
                            are_files_complete = False
                            break
                    if not are_files_complete:
                        break
            if are_files_complete:
                print("We already have all the value files for this setup. Skipping...")
                
            # open the PhotoZ file
            if enable_photoZ_interaction and not is_zda_file_open and not are_files_complete:
                aPhz.select_PhotoZ()
                aPhz.open_zda_file(subdir + "/" + zda_file)
                is_zda_file_open = True

            # open roi file in photoZ
            if enable_photoZ_interaction and not are_files_complete:
                aPhz.select_roi_tab()
                aPhz.open_roi_file(subdir + "/" + roi_file)

                
            for stim_time_idx in range(1,3):
                stim_time = [stim_time_1, stim_time_2][stim_time_idx-1]
        
                # set the stim window
                if enable_photoZ_interaction and not are_files_complete:
                    # is_measure_window_width_set is equal to window_width if window not yet set.
                    aPhz.set_measure_window(stim_time - pre_window_offset, 
                                            is_measure_window_width_set)
                    is_measure_window_width_set = None  # no need to set in future


                # save trace values from PhotoZ
                for trace_val_type in trace_values_to_save:
                    value_filename = subdir + "/" + trace_values_to_save[trace_val_type]['filename_id'] \
                                            + str(stim_time_idx) + "_" + filename_end
                    # may be able to skip saving file
                    if not (os.path.exists(value_filename) and not overwrite_intermed_dat_files):
                        value_method = trace_values_to_save[trace_val_type]['method']
                        if enable_photoZ_interaction and not are_files_complete:
                            value_method()
                            aPhz.save_trace_values(value_filename)
                    trace_values_to_save[trace_val_type]['full_path_filename_' + str(stim_time_idx)] = value_filename

            ################################
            new_rows = [{"Date" : date, 
                        "Slice_Loc_Rec": rec_id,
                        "ROI ID": i,
                        "Stim Time #1": stim_time_1,
                        "Stim Time #2": stim_time_2,
                        "Pixel Count": n_pixels[i - roi_id_start]}
                        for i in range(roi_id_start, roi_id_end+1)]

            for trace_val_type in trace_values_to_save:
                for stim_time_idx in range(1,3):
                    intermed_df = pd.read_csv(trace_values_to_save[trace_val_type]['full_path_filename_' + str(stim_time_idx)],
                                              sep='\t',
                                              header=None,
                                              names=['Index',  'Values'])
                    for i in range(len(new_rows)):
                        v = intermed_df['Values'][i]
                        new_rows[i][trace_val_type + " #" + str(stim_time_idx)] = v

            for new_row in new_rows:
                df = df.append(new_row, ignore_index=True)




scraping data from  02_01_02.zda using 02-01_ROIs_1_to_26.dat on date: 9/2/2022
We already have all the value files for this setup. Skipping...


scraping data from  02_01_03.zda using 02-01_ROIs_1_to_26.dat on date: 9/2/2022
We already have all the value files for this setup. Skipping...


scraping data from  02_02_01.zda using 02-02_ROIs_1_to_19.dat on date: 9/2/2022
We already have all the value files for this setup. Skipping...


scraping data from  02_02_02.zda using 02-02_ROIs_1_to_19.dat on date: 9/2/2022
We already have all the value files for this setup. Skipping...


scraping data from  02_02_03.zda using 02-02_ROIs_1_to_19.dat on date: 9/2/2022
We already have all the value files for this setup. Skipping...


scraping data from  03_01_03.zda using 03-01_ROIs_1_to_63.dat on date: 9/2/2022
We already have all the value files for this setup. Skipping...


scraping data from  03_02_03.zda using 03-02_ROIs_1_to_91.dat on date: 9/2/2022
We already have all the value files for th

In [17]:
# divide amplitudes by 1000
df['Max Amp #1'] /= 1000
df['Max Amp #2'] /= 1000

df['Interpulse Interval'] =  (df['Stim Time #2'] - df['Stim Time #1']) / df['Stim Time #2']

# add paired pulse ratio column
df['Paired Pulse Ratio'] = df['Max Amp #2'] / df['Max Amp #1']

# write the pandas dataframe back to csv
df.to_csv(output_csv, index=False)

In [18]:
df

,Date,Slice_Loc_Rec,ROI ID,Stim Time #1,Stim Time #2,Max Amp #1,Max Amp #2,Max Amp / SD #1,Max Amp / SD #2,Half Amp Latency #1,...,Half Width #2,Max Amp Latency #1,Max Amp Latency #2,Half Rise Time #1,Half Rise Time #2,Half Decay Time #1,Half Decay Time #2,Pixel Count,Paired Pulse Ratio,Interpulse Interval
0,9/2/2022,02_01_02,1,90,190,2.292330e-12,1.055700e-12,7.55451,3.47912,54.5739,...,8.20406,56.5,114.0,1.926140,6.870760,1.78735,1.33331,1,0.460536,0.526316
1,9/2/2022,02_01_02,2,90,190,2.133680e-12,1.637900e-12,5.37460,4.12576,51.8888,...,9.00727,53.0,105.0,1.111210,6.571470,4.15367,2.43580,1,0.767641,0.526316
2,9/2/2022,02_01_02,3,90,190,2.289920e-12,9.539350e-13,3.93704,1.64009,54.5192,...,3.23661,55.5,106.0,0.980806,1.589790,1.57432,1.64682,1,0.416580,0.526316
3,9/2/2022,02_01_02,4,90,190,1.676650e-12,9.525740e-13,4.77938,2.71537,45.6815,...,5.12062,57.0,102.5,11.318500,3.537700,2.19128,1.58292,1,0.568141,0.526316
4,9/2/2022,02_01_02,5,90,190,1.503100e-12,1.296050e-12,3.88842,3.35279,54.4136,...,7.29437,55.5,105.5,1.086450,3.058080,1.55683,4.23629,2,0.862251,0.526316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559,9/2/2022,07_01_03,96,90,290,4.363260e-03,3.807220e-03,4.79678,4.18550,53.5368,...,4.19288,55.0,155.0,1.463240,1.798700,1.95136,2.39418,1,0.872563,0.689655
2560,9/2/2022,07_01_03,97,90,290,3.660680e-03,2.254360e-03,6.30503,3.88282,52.1133,...,2.69060,54.5,155.0,2.386690,1.004250,2.99968,1.68635,2,0.615831,0.689655
2561,9/2/2022,07_01_03,98,90,290,4.659140e-03,3.549770e-03,9.88312,7.52989,53.5652,...,4.20256,55.0,154.0,1.434750,1.216380,1.69815,2.98619,1,0.761894,0.689655
2562,9/2/2022,07_01_03,99,90,290,4.001890e-03,2.908220e-03,5.56659,4.04531,49.5994,...,2.61035,50.5,150.5,0.900588,0.714877,6.15737,1.89547,2,0.726712,0.689655
